In [1]:
#inicializando GEE
try:
    import ee
except ModuleNotFoundError: 
    !pip install earth-engine-api
    import ee
    
try:
    ee.Initialize()
except ee.EEException:
    !earthengine authenticate
    
import geemap
import folium
from ipygee import *
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd

In [3]:
id467_MODIS = r"C:\Users\guilherme.fronza\OneDrive\cacauFlorestaAcelerador\amazon_acelerador\04a_dados_espaciais\02_dados_scripts\source\ID_467_MODIS_2.shp"
#create ee.Feature with GEEMAP
pol = geemap.shp_to_ee(id467_MODIS)

#Modis scale factor function
def scale_factor_MODIS(image):
    # scale factor for the MODIS MOD13Q1 product
    return image.multiply(0.0001).copyProperties(image, ['system:time_start'])

#plot all timeseries
def TS_NDVI_plot(imagecoll, polygon, scale):
    #time series
    polygon_ndvi = chart.Image.series(**{'imageCollection': imagecoll.select('NDVI'),
                                       'region': polygon,
                                       'reducer': ee.Reducer.mean(),
                                       'scale': scale,
                                       'xProperty': 'system:time_start'})
    df = polygon_ndvi.dataframe
    return df

#collections 2015 - 2021
MOD13Q1 = (ee.ImageCollection('MODIS/006/MOD13Q1')
                .filterBounds(pol)
                .filterDate(ee.DateRange('2000-02-10','2022-12-31'))
                .sort('system:time_start'))
#NDVI selection and scale factor to MODIS
ndvi_modis = MOD13Q1.select('NDVI')

# mapping function to multiply by the scale factor
images_ndvi_modis_scaled = ndvi_modis.map(scale_factor_MODIS)

MOD13Q1_NDVI_ts = TS_NDVI_plot(images_ndvi_modis_scaled, pol, scale=250)

In [4]:
MOD13Q1_NDVI_ts

,NDVI
2000-02-18,0.8816
2000-03-05,0.8043
2000-03-21,0.7373
2000-04-06,0.4459
2000-04-22,0.8703
...,...
2021-12-19,0.8739
2022-01-01,0.1349
2022-01-17,0.8558
2022-02-02,0.8456


In [5]:
MOD13Q1_NDVI_ts.to_excel("MOD13Q1_NDVI_ts.xlsx",
             sheet_name='MOD13Q1_NDVI_ts')  